In [1]:
import torch
import torch_geometric
from torch_geometric.data import Data

In [17]:
# random matrix -- variational/ weights
A = [[0.52, 0.89, 0.23, 0.67],
 [0.11, 0.45, 0.76, 0.98],
 [0.34, 0.21, 0.87, 0.05],
 [0.71, 0.33, 0.62, 0.14]]

A = torch.tensor(A, dtype=torch.float32)
print(A)
print(torch.sigmoid(A))
print((A > 0.5).float())

tensor([[0.5200, 0.8900, 0.2300, 0.6700],
        [0.1100, 0.4500, 0.7600, 0.9800],
        [0.3400, 0.2100, 0.8700, 0.0500],
        [0.7100, 0.3300, 0.6200, 0.1400]])
tensor([[0.6271, 0.7089, 0.5572, 0.6615],
        [0.5275, 0.6106, 0.6814, 0.7271],
        [0.5842, 0.5523, 0.7047, 0.5125],
        [0.6704, 0.5818, 0.6502, 0.5349]])
tensor([[1., 1., 0., 1.],
        [0., 0., 1., 1.],
        [0., 0., 1., 0.],
        [1., 0., 1., 0.]])


In [3]:
x = torch.zeros(1,2, dtype=torch.float64)
x

tensor([[0., 0.]], dtype=torch.float64)

In [2]:
edge_index = torch.tensor([[0, 1, 1, 2], [1, 0, 2, 1]], dtype=torch.long)
num_nodes = 3
data = Data(edge_index=edge_index, num_nodes=num_nodes)
print(data)

Data(edge_index=[2, 4], num_nodes=3)


In [11]:
u, v = int(edge_index[0, 2]), int(edge_index[1, 2])
print(u, v)

1 2


In [6]:
from torch_geometric.utils import to_dense_adj
A = to_dense_adj(edge_index)

In [5]:
adj_matrix = torch.zeros((data.num_nodes, data.num_nodes), dtype=torch.long)
for i in range(data.edge_index.size(1)):
    u, v = int(data.edge_index[0, i]), int(data.edge_index[1, i])
    adj_matrix[u, v] = 1
    adj_matrix[v, u] = 1

print(adj_matrix)


tensor([[0, 1, 0],
        [1, 0, 1],
        [0, 1, 0]])


In [13]:
adj_matrix[v]

tensor([0, 1, 0])

In [7]:
adj_squared_u = torch.mm(adj_matrix, adj_matrix)

adj_row_u = adj_matrix[u]
adj_row_v = adj_matrix[v]

bitwise_and_result = torch.bitwise_and(adj_squared_u[u], adj_row_v)
sum_bitwise_and_result = torch.sum(bitwise_and_result)

sum_adj_u = torch.sum(adj_row_u)
sum_adj_v = torch.sum(adj_row_v)

neg_index = sum_bitwise_and_result.float() / (sum_adj_u + sum_adj_v)

print("Neg Index for edge (u, v):", neg_index.item())

Neg Index for edge (u, v): 0.6666666865348816


optimization over complete list of PRE-COMPUTE sigma_1 [contains neg_index for all edges] and sigma_2[]

In [ ]:
import torch
from torch_geometric.data import Data
import torch.optim as optim

# Define neg_index, pos_index, custom_objective as provided

# Define a function to create a variational matrix by optimizing the custom objective
def create_variational_matrix(data, num_nodes, edge_index):
    # Initialize parameters for the variational matrix
    variational_matrix = torch.nn.Parameter(torch.rand((num_nodes, num_nodes), dtype=torch.float))

    # Define an optimizer for the parameters of the variational matrix
    optimizer = optim.Adam([variational_matrix], lr=0.01)

    # Optimization loop
    for epoch in range(1000):  # Adjust the number of epochs as needed
        optimizer.zero_grad()

        # Calculate sigma_1 and sigma_2 using the provided functions
        sigma_1 = torch.tensor(neg_index(data))
        sigma_2 = torch.tensor(pos_index(data))

        # Calculate the custom objective function
        objective = custom_objective(sigma_1 - 0.5, sigma_2)

        # Minimize the objective function
        loss = torch.min(objective)
        loss.backward()
        optimizer.step()

        if epoch % 100 == 0:
            print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

    # Sampling from the learned variational matrix to generate an adjacency matrix
    adjacency_matrix = torch.sigmoid(variational_matrix)  # Sigmoid to constrain values between 0 and 1

    # Thresholding to convert probabilities into binary adjacency matrix
    adjacency_matrix_binary = (adjacency_matrix > 0.5).float()

    return adjacency_matrix_binary

# Create a sample data object (Replace this with your actual data)
num_nodes = 10
edge_index = torch.tensor([[0, 1, 1, 2, 2, 3],
                           [1, 0, 2, 1, 3, 2]], dtype=torch.long)
data = Data(num_nodes=num_nodes, edge_index=edge_index)

# Create the variational matrix and sample an adjacency matrix
generated_adjacency_matrix = create_variational_matrix(data, num_nodes, edge_index)

print("Generated Adjacency Matrix:")
print(generated_adjacency_matrix)


Optimization for [PRE COMPUTE / RECURRENT CALLING] each pair of u,v in the edge list

In [ ]:
import torch
from torch_geometric.data import Data
import torch.optim as optim

# Define neg_index, pos_index, custom_objective as provided

# Define a function to create a variational matrix by optimizing the custom objective
def create_variational_matrix(data, num_nodes, edge_index):
    # Initialize parameters for the variational matrix
    variational_matrix = torch.nn.Parameter(torch.rand((num_nodes, num_nodes), dtype=torch.float))

    # Define an optimizer for the parameters of the variational matrix
    optimizer = optim.Adam([variational_matrix], lr=0.01)

    # Optimization loop
    for epoch in range(1000):  # Adjust the number of epochs as needed
        optimizer.zero_grad()

        # Calculate sigma_1 and sigma_2 using the provided functions
        sigma_1 = torch.tensor(neg_index(data))
        sigma_2 = torch.tensor(pos_index(data))

        # Calculate the custom objective function
        objective = custom_objective(sigma_1 - 0.5, sigma_2)

        # Minimize the objective function
        loss = torch.min(objective)
        loss.backward()
        optimizer.step()

        if epoch % 100 == 0:
            print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

    # Sampling from the learned variational matrix to generate an adjacency matrix
    adjacency_matrix = torch.sigmoid(variational_matrix)  # Sigmoid to constrain values between 0 and 1

    # Thresholding to convert probabilities into binary adjacency matrix
    adjacency_matrix_binary = (adjacency_matrix > 0.5).float()

    return adjacency_matrix_binary

# Create a sample data object (Replace this with your actual data)
num_nodes = 10
edge_index = torch.tensor([[0, 1, 1, 2, 2, 3],
                           [1, 0, 2, 1, 3, 2]], dtype=torch.long)
data = Data(num_nodes=num_nodes, edge_index=edge_index)

# Create the variational matrix and sample an adjacency matrix
generated_adjacency_matrix = create_variational_matrix(data, num_nodes, edge_index)

print("Generated Adjacency Matrix:")
print(generated_adjacency_matrix)
